# Audio Synthesis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from scipy.signal import butter, sosfiltfilt, sosfreqz, freqs, spectrogram, sawtooth

In [ ]:
SAMPLE_RATE = 44_100  # See https://en.wikipedia.org/wiki/Sampling_(signal_processing)

# Create a signal

In [ ]:
duration = 1
frequency = 440

# Create a time window
t = np.linspace(0, duration, num=int(duration * SAMPLE_RATE))

# Create a signal using the time window and frequency
y = np.sin(...)


# Plot the signal

In [ ]:
# Plot a single cycle of our waveform
single_cycle_end = int(
    1/frequency * len(y)
)

single_cycle = y[:single_cycle_end]

plt.plot(single_cycle);

In [ ]:
# Plot multiple cycles
plt.plot(y[:single_cycle_end * 3]);

# Plot the frequency response curve

In [ ]:
def plot_frequency_response(y: np.ndarray):
    plt.magnitude_spectrum(y, Fs=SAMPLE_RATE, scale='dB')
    plt.title("Frequency Response")
    plt.show()

In [ ]:
plot_frequency_response(y)

# Play the signal

In [ ]:
# Helper function so we can play np.ndarray as audio
def convert_to_audio(y: np.ndarray) -> np.ndarray:
    y *= 32767 / np.max(np.abs(y))
    y = y.astype(np.int16)
    return y

In [ ]:
audio = convert_to_audio(y)
ipd.Audio(audio, rate=SAMPLE_RATE)

# Create an amplifier

### Create the attack envelope

In [ ]:
# First, create the attack envelope. 
# Increase the signal from over a given duration
attack = ...  # in seconds
attack_amp = np.linspace(...)

In [ ]:
# Combine the amplifier and the original signal
amped_y = ...

plt.plot(amped_y);

### Implement Decay, Sustain and Release

In [ ]:
# Now, do the same thing for Decay, Sustain and Release
# Create one single envelope and combine it with the original signal

### Combine all envelopes into one

In [ ]:
amp = np.concatenate([...])

### Amplify the original signal

In [ ]:
amped_y = ...

### Trim the audio (removing zeros at start and end)

In [ ]:
amped_y = np.trim_zeros(amped_y)

# Plot the amplified signal

In [ ]:
plt.plot(amped_y);

# Play the amplified audio

In [ ]:
audio = convert_to_audio(amped_y)
ipd.Audio(audio, rate=SAMPLE_RATE)

# Filtering the audio

In [ ]:
def plot_filter(sos, cutoff):
    w, h = sosfreqz(sos, worN=8000)

    x = w * SAMPLE_RATE * 1.0 / (2 * np.pi)
    y = 20 * np.log10(abs(h))
    plt.figure(figsize=(10,5))
    plt.semilogx(x, y)
    plt.plot(cutoff, -3, 'ko')
    plt.axvline(cutoff, color='k')
    plt.ylabel('Amplitude [dB]')
    plt.xlabel('Frequency [Hz]')
    plt.ylim((-80, 10))
    plt.xlim((20,20_000))
    plt.title('Frequency response')
    plt.grid(which='both', linestyle='-', color='grey')
    plt.xticks([20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000], ["20", "50", "100", "200", "500", "1K", "2K", "5K", "10K", "20K"])
    plt.show()



def butter_filter(x: np.ndarray, cutoff: float, order: int = 2, filter_type: str = 'lowpass', plot=False) -> np.ndarray:
    nyquist_frequency = 0.5 * SAMPLE_RATE
    normal_cutoff = cutoff / nyquist_frequency

    # Get the filter coefficients
    sos = butter(order, normal_cutoff, btype=filter_type, analog=False, output='sos')

    if plot:
        plot_filter(sos, cutoff)

    # apply the filter to the signal
    y = sosfiltfilt(sos, x)
    return y

In [ ]:
y = np.sin(t*frequency*2*np.pi)

In [ ]:
filter_frequency = 10000
filtered_y = butter_filter(y,  order=2, cutoff=filter_frequency, plot=True)

In [ ]:
audio = convert_to_audio(filtered_y)
ipd.Audio(audio, rate=SAMPLE_RATE)

In [ ]:
amp